In [ ]:
import itertools
from pathlib import Path

import matplotlib.pyplot as plt
import torch

from face_verification.preprocessing import (
    split_identities,
    find_unique_identities,
    find_paths_with_identities,
)
from face_verification.dataloaders import create_dataloader
from face_verification.losses import CrossEntropy
from face_verification.metrics import EmbeddingAccuracy
from face_verification.monitors import EarlyStopping, ModelCheckpoint
from face_verification.models.face_verification import ArcFace, MobileNetV2
from face_verification.trainers import ArcFaceTrainer

In [ ]:
IMAGE_DIR = Path("meglass/")
CHECKPOINT_DIR = Path("checkpoint/")
VALID_SIZE = 0.1
TEST_SIZE = 0.1
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 8
RANDOM_STATE = 12
N_EPOCHS = 100
LEARNING_RATE = 1e-5

In [ ]:
image_paths = list(IMAGE_DIR.glob("*.jpg"))

In [ ]:
unique_identities = find_unique_identities(image_paths)

In [ ]:
train_identities, test_identities, valid_identities = split_identities(
    unique_identities, VALID_SIZE, TEST_SIZE, RANDOM_STATE
)

In [ ]:
train_identities = train_identities[:100]

In [ ]:
train_paths = find_paths_with_identities(image_paths, train_identities)
test_paths = find_paths_with_identities(image_paths, test_identities)
valid_paths = find_paths_with_identities(image_paths, valid_identities)

In [ ]:
train_loader = create_dataloader(
    train_paths, IMAGE_SIZE, BATCH_SIZE, train=True
)
test_loader = create_dataloader(
    test_paths, IMAGE_SIZE, BATCH_SIZE, train=False
)
valid_loader = create_dataloader(
    valid_paths, IMAGE_SIZE, BATCH_SIZE, train=False
)

In [ ]:
print(len(train_loader))
print(len(test_loader))
print(len(valid_loader))

In [ ]:
train_iter = iter(train_loader)
image_batch, label_batch = next(train_iter)

In [ ]:
print(image_batch.shape)
print(label_batch.shape)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(10, 5))
for axis, image in zip(axes.flatten(), image_batch.permute(0, 2, 3, 1)):
    axis.imshow(image)

In [ ]:
arcface = ArcFace(len(train_identities))
embedder = MobileNetV2()

In [ ]:
trainable_params = [
    param for param in
    itertools.chain(arcface.parameters(), embedder.parameters())
    if param.requires_grad
]

In [ ]:
loss = CrossEntropy()
optimizer = torch.optim.Adam(trainable_params, LEARNING_RATE)
metrics = [EmbeddingAccuracy()]
monitors = [
    ModelCheckpoint(loss, CHECKPOINT_DIR),
    EarlyStopping(loss, patience=10),
]

In [ ]:
trainer = ArcFaceTrainer(
    arcface, embedder, loss, optimizer, metrics, monitors
)

In [ ]:
trainer.train(train_loader, valid_loader, N_EPOCHS)

In [ ]:
trainer.test(test_loader)